In [32]:
import sys
sys.path.append('../pyuqtkarray/')
sys.path.append('../pce/')
sys.path.append('../quad/')
sys.path.append('../tools/')
sys.path.append('../lowrank/')

# Import Modules
import PyUQTk.uqtkarray as uqtkarray
import PyUQTk.quad as uqtkquad
import PyUQTk.pce as uqtkpce
import PyUQTk.tools as uqtktools
import PyUQTk.lowrank as lr
import PyUQTk.pce as uqtkpce

import numpy as np

from scipy.stats import qmc
import PyUQTk.PyPCE.pce_tools as pce_tools

In [23]:
#xfile = xdata.dat
#yfile = ydata.dat
basistype = "PC"
intpar = 4
rank = 4
maxiter = 4
strpar = 50
meth = "lsq"

lflag=False
vflag=False

nx = 10
ndim = 2

In [37]:
rng = qmc.LatinHypercube(d=ndim, seed=42)


x_np = 2*rng.random(n=nx)-1
poly=uqtkpce.PCSet("NISPnoq", rank, ndim, 'LU', 1.0, 0.0)
coef=np.array([i+1 for i in range(poly.GetNumberPCTerms())])
y_np = pce_tools.UQTkEvaluatePCE(poly, coef, x_np)

In [38]:
# empty array for training data
xdata=uqtkarray.dblArray2D(nx, ndim) #Nxd training samples
xdata=uqtkarray.numpy2uqtk(np.asfortranarray(x_np))

#Read true function evaluation
ydata=uqtkarray.dblArray1D(nx,0.e0)
ydata=uqtkarray.numpy2uqtk(np.asfortranarray(y_np))

yapprox=uqtkarray.dblArray1D(nx, 0.e0) # keep empty
error=uqtkarray.dblArray1D(nx, 0.e0)   # keep empty

#Sanity checks
ny = ydata.XSize()
if(nx != ny):
    print("Number of input samples is not the same as output samples. Exiting.\n")

In [ ]:
## Declare tensor object
f = lr.CanonicalTensor()

In [39]:
####### FunctionalBases*--how can i find out what this is
####### Array1D<string> --is there a function that allows me to create this in python

## Initialize an array of functional bases
testFArr1=uqtkarray.intArray1D()
#Array1D<FunctionalBases* > testFArr(1)
order = 4

# Set type of basis (PC or PL) in each dimension
#FunctionalBases *newBase;

if (basistype=="PC"):
    baseOrder = uqtkarray.intArray1D(ndim,order)
    pctype = uqtkarray.intArray1D(ndim, strpar)
    newBase = lr.PCBases(pctype, baseOrder)
    testFArr[0] = newBase
    
# elif(basistype=="POL"):
#     baseOrder=uqtkarray.intArray(ndim,order)
#     newBase = lr.PLBases(baseOrder)
#     testFArr0 = newBase
# else:
#     print("Basistype %s is not recognized, should be PC or POL.",basistype)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. _lowrank.PCBases(arg0: Array1D<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, arg1: _uqtkarray.intArray1D)

Invoked with: <_uqtkarray.intArray1D object at 0x7ff1edfdc730>, <_uqtkarray.intArray1D object at 0x7ff1ee419fb0>

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.

In [ ]:
### what is testFArr--it has FunctionalBases and newbase in it
# it an array with the length of the prder of the CanonicalTensor

In [ ]:
# Initialize algorithm
s = lr.CanonicalTensorALSLeastSquares()
s.setBases(testFArr)
s.setIteration(maxiter)


#Set the regularization parameter
if (lflag):
    lmbda = dblpar
    s.setLambda(lmbda)

# Loop over one to rank
for r in range(1, rank+1):
    s.setRank(r)
    f = s.solveDirect(xdata, ydata, vflag) # gives info back to CT, f

    fun = lr.FunctionalTensor(f,testFArr) # makes a functional tensor, fun

yapprox = fun.tensorEval(xdata)
error = subtract(ydata,yapprox)

print("Norm of the error for rank %d approximation = %e\n",r,(norm(error)/norm(ydata)));